# PrioritizeMe

Aplikacja do priorytetowego planowania zadań to narzędzie, które pomoże efektywnie zarządzać czasem i zadaniami. Dzięki niej możesz tworzyć zadania, przypisywać im tytuły oraz opisy, a następnie nadawać im priorytety w zależności od ich ważności. 

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania wszystkich zadań, umożliwia podstawowe operacje *CRUD* (*Create-Remove-Update-Delete*), oraz ich filtrowanie i sortowanie.

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" /></td><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania zadań, oraz jako *SSOT* (*Single Source Of Truth*)
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę zadań, umożliwia usunięcie, oznaczenie zadania jako wykonanego, oraz szybką zmianę nadanego priorytetu
    - ekran dodania zadania
    - ekran aktualizacji zadania, który umożliwia również usunięcie
- Aplikacja wykorzystuje `Jetpack Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w głównek aktywności w celu uzyskania współdzielenia jego instancji przez wszystkie fragmenty


Dodajmy wymagane zależności do projektu

```verbatim
>>> Build.gradle(Project)
```

```kotlin
// Top-level build file where you can add configuration options common to all sub-projects/modules.
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath ("androidx.navigation:navigation-safe-args-gradle-plugin:2.5.3")
    }
}
plugins {
    id("com.android.application") version "8.1.0" apply false
    id("org.jetbrains.kotlin.android") version "1.8.0" apply false
}
```

```verbatim
>>> Build.gradle(Module)
```

```kotlin
plugins {
    id("com.android.application")
    id("org.jetbrains.kotlin.android")
    id("kotlin-kapt")
    id("androidx.navigation.safeargs.kotlin")
}
android {
    ...
    buildFeatures {
        viewBinding = true
    }
    compileOptions {
        sourceCompatibility = JavaVersion.VERSION_17
        targetCompatibility = JavaVersion.VERSION_17
    }
    kotlinOptions {
        jvmTarget = "17"
    }
}
dependencies {

    implementation("androidx.room:room-ktx:2.5.2")
    annotationProcessor("androidx.room:room-compiler:2.5.2")
    implementation("androidx.room:room-ktx:2.5.2")
    kapt("androidx.room:room-compiler:2.5.2")

    implementation ("androidx.navigation:navigation-fragment:2.5.3")
    implementation ("androidx.navigation:navigation-ui:2.5.3")

    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
    // LiveData
    implementation ("androidx.lifecycle:lifecycle-livedata-ktx:2.6.1")
    //Fragment
    implementation ("androidx.fragment:fragment-ktx:1.6.1")

    implementation("androidx.cardview:cardview:1.0.0")

    implementation ("androidx.recyclerview:recyclerview:1.3.1")
    ...
}
```

W pierwszym kroku przygotujmy dane, które wykorzystamy jako *dummy data* do inicjalizacji bazy danych.

In [ ]:
data class Task(val title: String, val description: String)

In [ ]:
object DataProvider {

    private val titles = listOf(
        "Dokończ Raport",
        "Przygotuj Prezentację",
        "Zadzwoń do Klienta",
        "Aktualizuj Zawartość Strony",
        "Weź Udział w Spotkaniu Zespołu",
        "Przejrzyj Kod",
        "Wysyłka Faktur",
        "Badanie Trendów na Rynku",
        "Plan Kampanii Marketingowej",
        "Organizuj Pliki",
        "Przygotuj Propozycję Budżetu",
        "Twórz Mockupy Produktu",
        "Planuj Posty na Mediach Społecznościowych",
        "Spotkanie z Zespołem Projektowym",
        "Testuj Nowe Oprogramowanie",
        "Dochodź do Potencjalnych Klientów",
        "Projektuj Logo",
        "Napisz Post na Blogu",
        "Ustal Harmonogram Projektu",
        "Przejrzyj Specyfikacje Produktu",
        "Organizuj Plan Podróży",
        "Przeprowadź Ankietę Użytkowników",
        "Analizuj Dane Sprzedażowe",
        "Przygotuj Materiały Szkoleniowe",
        "Koordynuj Logistykę Wydarzenia",
        "Burza Mózgów nad Nowymi Pomysłami",
        "Optymalizuj SEO Strony",
        "Przeprowadź Oceny Wyników Pracowników",
        "Twórz Prototypy Aplikacji"
    )

    private val descriptions = listOf(
        "Ukończ końcowy raport dotyczący wyników kwartalnych.",
        "Stwórz prezentację PowerPoint na nadchodzące spotkanie z klientem.",
        "Skontaktuj się z klientem w celu dalszej rozmowy.",
        "Aktualizuj treść na stronie głównej witryny internetowej.",
        "Zorganizuj logistykę nadchodzącego wydarzenia firmowego.",
        "Burz mózg nad innowacyjnymi pomysłami na poprawę produktu.",
        "Optymalizuj SEO strony internetowej dla lepszych wyników w wyszukiwarkach.",
        "Przeprowadź oceny wyników pracowników zespołu.",
        "Twórz interaktywne prototypy dla nowej aplikacji.",
        "Dokładnie przetestuj i oceniaj funkcjonalność nowego oprogramowania. Dokumentuj wszelkie błędy lub problemy i współpracuj blisko z zespołem developerskim, aby zapewnić ich szybkie rozwiązanie.",
        "Kontynuuj kontakt z potencjalnymi klientami i odpowiedz na zapytania wygenerowane przez kampanie marketingowe i wydarzenia networkingowe. Udziel spersonalizowanych odpowiedzi i dostarcz informacji dostosowanych do potrzeb każdego potencjalnego klienta.",
        "Zaprojektuj nowe logo dla przemiany marki, biorąc pod uwagę tożsamość marki, grupę docelową i trendy w branży. Przedstaw wiele opcji logo w celu uzyskania opinii i wybierz ostateczny projekt.",
        "Napisz dobrze przemyślany i informacyjny post na blogu na temat najnowszych wydarzeń w branży. Wykorzystaj dane, statystyki i wiedzę ekspertów, aby dostarczyć cenne treści czytelnikom bloga.",
        "Ustal harmonogram dla nadchodzących faz projektu, uwzględniając dostępność zasobów, zależności i potencjalne ryzyka. Podziel się harmonogramem z interesariuszami projektu w celu uzyskania opinii.",
        "Przejrzyj i sfinalizuj specyfikacje produktu, upewniając się, że są one kompleksowe i zgodne z celami projektu. Podziel się specyfikacjami z zespołem developerskim w celu wdrożenia.",
        "Zaplanuj plan podróży służbowej, uwzględniając rozkłady lotów, zakwaterowanie, transport i organizację spotkań. Podziel się planem podróży ze wszystkimi odpowiednimi członkami zespołu.",
        "Przeprowadź szczegółową ankietę, aby pozyskać opinię użytkowników na temat użyteczności produktu, funkcji i ogólnego zadowolenia. Analizuj wyniki ankiety, aby zidentyfikować obszary do poprawy.",
        "Analizuj dane sprzedażowe i identyfikuj trendy w celu podejmowania decyzji opartych na danych dotyczących strategii sprzedaży i poprawek produktowych. Przygotuj obszerny raport, aby przedstawić wyniki zespołowi sprzedażowemu i zarządowi.",
        "Przygotuj materiały szkoleniowe dla nowych pracowników, w tym przewodniki wprowadzające, moduły szkoleniowe i prezentacje. Upewnij się, że materiały obejmują wszystkie istotne aspekty roli nowego pracownika.",
        "Koordynuj logistykę nadchodzącego wydarzenia firmowego, takiego jak konferencja czy wyjazd integracyjny zespołu. Zabezpiecz lokalizacje, catering, transport i wszelkie niezbędne wyposażenie.",
        "Burz mózg nad innowacyjnymi pomysłami na poprawę produktu, uwzględniając opinię użytkowników, trendy rynkowe i postęp technologiczny. Przedstaw pomysły zespołowi rozwoju produktu w celu oceny.",
        "Optymalizuj SEO strony internetowej, aby poprawić pozycje w wynikach wyszukiwania i zwiększyć ruch organiczny. Przeprowadź badania słów kluczowych, aktualizuj meta tagi i wdroż na stronie najlepsze praktyki SEO.",
        "Przeprowadź oceny wyników pracowników, dostarczając konstruktywną opinię i wyznaczając cele rozwoju osobistego i zawodowego. Doceniaj i nagradzaj wyjątkowe osiągnięcia.",
        "Twórz interaktywne prototypy dla nowej aplikacji, uwzględniając przyjazny dla użytkownika design i intuicyjną nawigację. Testuj prototypy z potencjalnymi użytkownikami, aby uzyskać opinię i walidację."
    )

    val tasks = (0..4).map { Task(
        title = titles.random(),
        description = descriptions.random(),
        priority = Priority.values().random()
    ) }
}

## Baza danych

Model zadania zawiera pola
-  `title: String` - tytuł zadania
-  `description: String` - opis zadania
-  `isDone: Boolean = false` - flaga określająca czy zadanie zostało wykonane
-  `priority: Priority = Priority.NORMALNY` - priorytet zadania

In [ ]:
@Entity(tableName = "task_table")
data class Task(
    @PrimaryKey(autoGenerate = true) val id: Int = 0,
    val title: String,
    val description: String,
    val isDone: Boolean = false,
    val priority: Priority = Priority.NORMALNY
)

Priorytet zdefiniujemy w klasie `enum`

In [ ]:
enum class Priority {
    WYSOKI,
    NORMALNY,
    NISKI,
    WYKONANY
}

Dodajmy interfejs `dao` zawierający definicje operacji na bazie danych. 

In [ ]:
@Dao
interface TaskDao {

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertTask(task: Task)

    @Insert(onConflict = OnConflictStrategy.IGNORE)
    suspend fun insertAllTasks(tasks: List<Task>)

    @Query("SELECT * FROM task_table ORDER BY id ASC")
    fun getTasks(): Flow<List<Task>>

    @Delete
    suspend fun deleteTask(task: Task)

    @Update
    suspend fun updateTask(task: Task)

    @Query("DELETE FROM task_table")
    suspend fun deleteAll()
}

Dodajmy repozytorium z odpowiednimi metodami.

In [ ]:
class TaskRepository(private val application: Application) {
    private val db = TaskDatabase.getDatabase(application)
    private val dao = db.taskDao()

    fun getTasks() = dao.getTasks()
    suspend fun insertTask(task: Task) = dao.insertTask(task)
    suspend fun insertAllTasks(tasks: List<Task>) = dao.insertAllTasks(tasks)
    suspend fun deleteTask(task: Task) = dao.deleteTask(task)
    suspend fun updateTask(task: Task) = dao.updateTask(task)
    suspend fun deleteAll() = dao.deleteAll()
}

Następnie dodajmy `ViewModel`, który zarządza danymi związanymi z zadaniami, interakcjami z repozytorium (dostęp do danych) oraz logiką biznesową związaną z priorytetami i stanem zadań.

- tasksState - `StateFlow` z listą zadań. `stateIn` używa `viewModelScope` do obsługi cyklu życia i definiuje, jak strumień powinien być obsługiwany.
- Inicjalizacja bazy danych z danymi przykładowymi - w metodzie `init`, `reinitializeDatabaseWithDummyData()` jest wywołane w celu dodania przykładowych danych do bazy danych poprzez usuwanie istniejących zadań i dodawanie nowych, pochodzących z `DataProvider.tasks`.
- Operacje CRUD na bazie danych - metody `addAll`, `deleteAll`, `updateTask`, `insertTask` i `deleteTask` są wykorzystywane do interakcji z bazą danych poprzez repozytorium. Wszystkie działania są wykonywane wewnątrz bloku `viewModelScope.launch`, co oznacza, że są asynchroniczne i bezpieczne w kontekście wątków.
- Logika priorytetów i stanu zadań - pozostała część kodu zawiera logikę związaną z priorytetami i stanem zadań. Metody `getTaskColor`, `increasePriority`, `decreasePriority` i `donePriority` zmieniają priorytet oraz stan zadań w zależności od ich obecnych wartości.
- Funkcje pomocnicze - `getTask(id: Int)` pozwala na pobranie konkretnego zadania o podanym `id`. `getTaskColor(task: Task)` zwraca kolor w zależności od priorytetu zadania.

In [ ]:
class TaskViewModel(application: Application) : AndroidViewModel(application) {

    private val repository = TaskRepository(application)

    val tasksState: StateFlow<List<Task>> = repository.getTasks().stateIn(
        viewModelScope,
        SharingStarted.WhileSubscribed(),
        emptyList()
    )

    init {
        reinitializeDatabaseWithDummyData()
    }

    private fun reinitializeDatabaseWithDummyData(){
        deleteAll()
        addAll(DataProvider.tasks)
    }

    private fun addAll(tasks: List<Task>){
        viewModelScope.launch {
            repository.insertAllTasks(tasks)
        }
    }

    private fun deleteAll(){
        viewModelScope.launch {
            repository.deleteAll()
        }
    }

    fun updateTask(task: Task){
        viewModelScope.launch {
            repository.updateTask(task)
        }
    }

    fun getTask(id: Int) =
        tasksState.value.find { it.id == id } ?: Task(title = "", description = "")

    fun addTask(task: Task){
        viewModelScope.launch {
            repository.insertTask(task)
        }
    }

    fun deleteTask(task: Task){
        viewModelScope.launch{
            repository.deleteTask(task)
        }
    }

    fun getTaskColor(task: Task): Color {
        return when(task.priority) {
            Priority.WYSOKI -> Color.valueOf(Color.RED)
            Priority.NISKI -> Color.valueOf(Color.BLUE)
            Priority.NORMALNY -> Color.valueOf(Color.GREEN)
            else -> Color.valueOf(128f, 203f, 196f)
        }
    }

    fun increasePriority(task: Task) {
        val nextPriority = when (task.priority) {
            Priority.WYSOKI -> Priority.WYSOKI
            Priority.NORMALNY -> Priority.WYSOKI
            Priority.NISKI -> Priority.NORMALNY
            Priority.WYKONANY -> Priority.WYKONANY
        }

        updateTask(task.copy(priority = nextPriority))
    }

    fun decreasePriority(task: Task) {
        val nextPriority = when (task.priority) {
            Priority.WYSOKI -> Priority.NORMALNY
            Priority.NORMALNY -> Priority.NISKI
            Priority.NISKI -> Priority.NISKI
            Priority.WYKONANY -> Priority.WYKONANY
        }

        updateTask(task.copy(priority = nextPriority))
    }

    fun donePriority(task: Task) {
        val nextPriority = when (task.priority) {
            Priority.WYSOKI -> Priority.WYKONANY
            Priority.NORMALNY -> Priority.WYKONANY
            Priority.NISKI -> Priority.WYKONANY
            Priority.WYKONANY -> Priority.WYKONANY
        }

        updateTask(task.copy(priority = nextPriority, isDone = true))
    }
}

Aplikacja posiada trzy ekrany, głównym jest lista wszystkich zadań. W nawigacji przejdziemy bezargumentowo na ekran dodania nowego zadania. Ekran aktualizacji zadania będzie wymagał przekazania `id` zadania, dzięki któremu załadujemy odpowiednie dane z listy. Sam `ViewModel` będzie **współdzielony** przez wszystkie ekrany, osiągniemy to poprzez jego zainicjowanie w komponencie `Navigation` i przekazanie tej instancji przez parametr do komponentów reprezentujących ekrany.

## Nawigacja

Dodajmy nawigację.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/taskListFragment">

    <fragment
        android:id="@+id/taskListFragment"
        android:name="com.example.prioritezmekotlin.ui.fragments.list.TaskListFragment"
        android:label="fragment_task_list"
        tools:layout="@layout/fragment_task_list" >
        <action
            android:id="@+id/action_taskListFragment_to_taskAddFragment"
            app:destination="@id/taskAddFragment" />
        <action
            android:id="@+id/action_taskListFragment_to_taskUpdateFragment"
            app:destination="@id/taskUpdateFragment" >
            <argument
                android:name="task_id"
                app:argType="integer"
                android:defaultValue="0" />
        </action>
    </fragment>
    <fragment
        android:id="@+id/taskAddFragment"
        android:name="com.example.prioritezmekotlin.ui.fragments.add.TaskAddFragment"
        android:label="fragment_task_add"
        tools:layout="@layout/fragment_task_add" >
        <action
            android:id="@+id/action_taskAddFragment_to_taskListFragment"
            app:destination="@id/taskListFragment" />
    </fragment>
    <fragment
        android:id="@+id/taskUpdateFragment"
        android:name="com.example.prioritezmekotlin.ui.fragments.edit.TaskUpdateFragment"
        android:label="fragment_task_update"
        tools:layout="@layout/fragment_task_update" >
        <action
            android:id="@+id/action_taskUpdateFragment_to_taskListFragment"
            app:destination="@id/taskListFragment" />
    </fragment>
</navigation>

Na Fragment przznaczony do aktualizacji zadania będziemy przekazywać argument - `id` zadania, więc do `action` dodajemy odpowiedni `argument`

```xml
<action
    android:id="@+id/action_taskListFragment_to_taskUpdateFragment"
    app:destination="@id/taskUpdateFragment" >
    <argument
        android:name="task_id"
        app:argType="integer"
        android:defaultValue="0" />
</action>
```

Będziemy wykorzystywać **współdzielony** `ViewModel`, polega to na zainicjowaniu instancji w aktywności hostującej i wykorzystanie odpowiedniego delegatu (`by activityViewModels`) we fragmentach. W ten sposób wszystkie fragmenty będą posiadały dokładnie tą samą instancję `ViewModel`.

In [ ]:
class MainActivity : AppCompatActivity() {

    val viewModel: TaskViewModel by viewModels()
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)
    }
}

## Ekrany

Przejdźmy do zaprojektowania layoutu fragmentu z listą.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:layout_margin="4dp"
    tools:context=".ui.fragments.list.TaskListFragment">

    <com.google.android.material.floatingactionbutton.FloatingActionButton
        android:id="@+id/addFab"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="24dp"
        android:layout_marginBottom="36dp"
        android:clickable="true"
        android:src="@drawable/ic_add"
        style="@style/ShapeAppearanceOverlay.Material3.FloatingActionButton"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:contentDescription="add" />

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recycler"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:layout_marginTop="16dp"
        android:layout_marginEnd="8dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Oprócz `RecyclerView` wykorzystujemy tutaj `FAB` (*Floating Action Button*). FAB jest przyciskiem umieszczonym w obszarze interfejsu, który zazwyczaj znajduje się na dolnej części ekranu. Jego celem jest dostarczenie szybkiego dostępu do kluczowych akcji lub funkcji aplikacji. Tutaj wykorzystamy ten przycisk aby umożliwić użytkownikowi przejście na ekran dodania zadania.

Będziemy wykorzystywać `RecyclerView`, więc potrzebujemy również layout dla pojedynczego elementu listy.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<com.google.android.material.card.MaterialCardView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="wrap_content"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    app:strokeColor="@color/black"
    android:background="#FF6200EE"
    app:cardCornerRadius="8dp">

    <LinearLayout
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:orientation="vertical"
        >

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:orientation="horizontal"
            >

            <TextView
                android:id="@+id/title"
                android:layout_width="0dp"
                android:layout_height="wrap_content"
                android:layout_weight="1"
                android:background="#B093E6"
                android:gravity="center"
                android:padding="4dp"
                android:text="Przeprowadź Ankietę Użytkowników"
                android:textColor="@color/black"
                android:textSize="16sp" />

            <ImageView
                android:id="@+id/priorityIcon"
                android:layout_width="wrap_content"
                android:layout_height="match_parent"
                android:layout_gravity="center"
                android:scaleType="center"
                android:paddingEnd="4dp"
                android:background="#B093E6"
                android:scaleX="1"
                android:scaleY="1"
                android:src="@drawable/ic_circle"
                android:contentDescription="TODO" />
        </LinearLayout>

        <TextView
            android:id="@+id/description"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:padding="4dp"
            android:text="Optymalizuj SEO strony internetowej, ..."
            android:textSize="14sp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/overviewRecyclerViewCarName" />

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:orientation="vertical">
            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="horizontal">

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/priorityUp"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="8dp"
                    android:layout_marginEnd="4dp"
                    app:icon="@drawable/ic_up"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/done"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="4dp"
                    android:layout_marginEnd="4dp"
                    app:icon="@drawable/ic_done"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/priorityDown"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_marginStart="4dp"
                    android:layout_marginEnd="4dp"
                    android:layout_height="wrap_content"
                    app:icon="@drawable/ic_down"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>

                <com.google.android.material.button.MaterialButton
                    android:id="@+id/delete"
                    style="?attr/materialButtonOutlinedStyle"
                    android:layout_width="0dp"
                    android:layout_weight=".25"
                    android:layout_marginStart="4dp"
                    android:layout_marginEnd="8dp"
                    android:layout_height="wrap_content"
                    app:icon="@drawable/ic_delete"
                    app:iconGravity="textStart"
                    app:iconPadding="0dp"/>
            </LinearLayout>

            <com.google.android.material.button.MaterialButton
                android:id="@+id/edit"
                style="?attr/materialButtonOutlinedStyle"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="8dp"
                android:layout_marginEnd="8dp"
                app:icon="@drawable/ic_edit"
                app:iconGravity="textStart"
                app:iconPadding="0dp" />

        </LinearLayout>

    </LinearLayout>
</com.google.android.material.card.MaterialCardView>

`ImageView` jest elementem, który zostanie wykorzystany w celu pokazania priorytetu zadania. Sam priorytet będziemy reprezentować za pomocą koloru. W tradycyjnym projektowaniu interfejsu (za pomocą plików xml i widoków) nie występuje `IconButton`, więc aby dodać przycisk tylko z ikoną (bez tekstu) musimy zasotować prosty trik.

```xml
app:icon="@drawable/ic_down"
app:iconGravity="textStart"
app:iconPadding="0dp"
```

Ustawiamy właściwość `iconGravity` na `textStart` aby umieścić ikonę przed tekstem, następnie ustawiamy `iconPadding` na `0dp`. W ten sposób, przy braku tekstu, osiągniemy przycisk z wyśrodkowaną ikoną.

Przejdźmy do implementacji klas wymaganych do obsługi `RecyclerView`.

In [ ]:
class TaskViewHolder(
    private val binding: RvItemBinding,
    private val onPriorityUp: (Task) -> Unit,
    private val onPriorityDown: (Task) -> Unit,
    private val onTaskDone: (Task) -> Unit,
    private val onDelete: (Task) -> Unit,
    private val onEdit: (Int) -> Unit,
) : RecyclerView.ViewHolder(binding.root) {
    fun bind(item: Task){
        binding.apply {
            title.text = item.title
            description.text = item.description
            priorityIcon.setColorFilter(priorityColor(item).toArgb(), PorterDuff.Mode.SRC_IN)
            priorityUp.setOnClickListener { onPriorityUp(item) }
            priorityDown.setOnClickListener { onPriorityDown(item) }
            done.setOnClickListener { onTaskDone(item) }
            delete.setOnClickListener { onDelete(item) }
            edit.setOnClickListener { onEdit(item.id) }
        }
    }

    private fun priorityColor(task: Task): Color {
        return when(task.priority) {
            Priority.WYSOKI -> Color.valueOf(Color.RED)
            Priority.NISKI -> Color.valueOf(Color.BLUE)
            Priority.NORMALNY -> Color.valueOf(Color.GREEN)
            else -> Color.valueOf(128f, 203f, 196f)
        }
    }
}

- Konstruktor klasy `TaskViewHolder`:
    - `binding` Przekazywany jest obiekt typu `RvItemBinding`, który reprezentuje powiązanie widoku elementu listy z odpowiednimi komponentami.
    - `onPriorityUp` Funkcja wywoływana po kliknięciu przycisku *Priority Up* w widoku elementu. Przekazywana jest tutaj funkcja lambda przyjmująca obiekt typu `Task`.
    - `onPriorityDown` Funkcja wywoływana po kliknięciu przycisku *Priority Down* w widoku elementu.
    - `onTaskDone` Funkcja wywoływana po kliknięciu przycisku *Done* w widoku elementu.
    - `onDelete` Funkcja wywoływana po kliknięciu przycisku *Delete* w widoku elementu.
    - `onEdit` Funkcja wywoływana po kliknięciu przycisku *Edit* w widoku elementu. Przekazywana jest tutaj funkcja lambda przyjmująca identyfikator (`Int`) zadania.
    

- Metoda `bind(item: Task)`:
    Metoda ta jest używana do wiązania danych przekazanych do widoku elementu.
    W tej metodzie następuje ustawienie tytułu i opisu zadania z obiektu `item` do odpowiednich komponentów interfejsu użytkownika.
    Kolor ikony priorytetu jest ustawiany na podstawie funkcji `priorityColor(item)` i przekazywany za pomocą `setColorFilter`.
    
- Prywatna metoda `priorityColor(task: Task): Color`:
    Ta metoda przyjmuje obiekt typu `Task` i zwraca kolor, który reprezentuje priorytet tego zadania.
    W zależności od wartości priorytetu zadania, metoda zwraca odpowiedni kolor (np. czerwony dla priorytetu *Wysoki*, niebieski dla *Niski* itp.).

In [ ]:
class TaskComparator : DiffUtil.ItemCallback<Task>() {
    override fun areItemsTheSame(oldItem: Task, newItem: Task): Boolean {
        return newItem === oldItem
    }

    override fun areContentsTheSame(oldItem: Task, newItem: Task): Boolean {
        return newItem == oldItem
    }
}

Klasa `TaskComparator` dziedziczy po klasie `DiffUtil.ItemCallback<Task>()` i jest wykorzystywana w celu porównywania i obliczania różnic między dwoma listami obiektów typu `Task`. W kontekście komponentu `RecyclerView`, `DiffUtil` pomaga zoptymalizować wydajność odświeżania listy poprzez identyfikację tylko tych elementów, które faktycznie uległy zmianie.

In [ ]:
class TaskAdapter(
    taskComparator: TaskComparator,
    private val onPriorityUp: (Task) -> Unit,
    private val onPriorityDown: (Task) -> Unit,
    private val onTaskDone: (Task) -> Unit,
    private val onDelete: (Task) -> Unit,
    private val onEdit: (Int) -> Unit,
): ListAdapter<Task, TaskViewHolder>(taskComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): TaskViewHolder {
        return TaskViewHolder(
            binding = RvItemBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            ),
            onPriorityUp = onPriorityUp,
            onPriorityDown = onPriorityDown,
            onTaskDone = onTaskDone,
            onDelete = onDelete,
            onEdit = onEdit
        )
    }

    override fun onBindViewHolder(holder: TaskViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item = item)
    }

}

- Konstruktor klasy `TaskAdapter`:
    - `taskComparator` Przekazywany jest obiekt klasy `TaskComparator`, który będzie wykorzystywany do porównywania obiektów w celu określenia zmian w liście.
    - Pozostałe parametry to lambdy i funkcje, które zostaną wywołane w odpowiedzi na interakcje użytkownika z elementami listy.
- Metoda `onCreateViewHolder`:
    Ta metoda tworzy nowy obiekt TaskViewHolder poprzez opakowanie widoku z layoutu RvItemBinding za pomocą LayoutInflater.
    Przekazywane są również funkcje i lambdy obsługujące interakcje związane z priorytetem, oznaczaniem jako zrobione, usuwaniem i edytowaniem zadania.
- Metoda `onBindViewHolder`:
    Ta metoda wiąże obiekt `TaskViewHolder` z odpowiednim obiektem `Task` z listy.
    Uzyskiwany jest element z listy za pomocą `getItem(position)`, a następnie przekazywany do metody `bind(item)`, aby uzupełnić widok danymi.

Przejdźmy do implementacji samego fragmentu

In [ ]:
class TaskListFragment : Fragment() {

    private lateinit var binding: FragmentTaskListBinding

    private val viewModel: TaskViewModel by activityViewModels()

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentTaskListBinding.inflate(inflater)

        val taskAdapter = TaskAdapter(
            taskComparator = TaskComparator(),
            onPriorityUp = { viewModel.increasePriority(it) },
            onPriorityDown = { viewModel.decreasePriority(it) },
            onTaskDone = { viewModel.donePriority(it) },
            onDelete = { viewModel.deleteTask(it) },
            onEdit = {
                val action = TaskListFragmentDirections.actionTaskListFragmentToTaskUpdateFragment(it)
                Navigation.findNavController(requireView()).navigate(action)
            }
        )
        viewLifecycleOwner.lifecycleScope.launch {
            viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
                viewModel.tasksState.collectLatest{ taskList ->
                    taskAdapter.submitList(taskList)
                }
            }
        }

        binding.recycler.apply {
            adapter = taskAdapter
            layoutManager = StaggeredGridLayoutManager(2, StaggeredGridLayoutManager.VERTICAL)
            itemAnimator = null
        }

        binding.addFab.setOnClickListener {
            val action = TaskListFragmentDirections.actionTaskListFragmentToTaskAddFragment()
            Navigation.findNavController(requireView()).navigate(action)
        }

        return binding.root
    }
}

Przejdźmy przez kod krok po kroku

In [ ]:
private val viewModel: TaskViewModel by activityViewModels()

- `ViewModel`: jest komponentem, który przechowuje i dostarcza dane związane z interfejsem użytkownika, takie jak stan, które nie powinno być przechowywane bezpośrednio w fragmencie czy aktywności, aby przetrwać zmiany konfiguracji (np. obrót ekranu).
- `ViewModel` *Sharing*: delegat `by activityViewModels()` to mechanizm, który umożliwia dzielenie tego samego `ViewModel`u między różnymi fragmentami (lub innymi komponentami) znajdującymi się w tym samym zakresie, np. w obrębie jednej aktywności. Oznacza to, że fragment `TaskListFragment` będzie używać `ViewModel`u, który jest dostępny w zakresie aktywności hostującej.

In [ ]:
val taskAdapter = TaskAdapter(
    taskComparator = TaskComparator(),
    onPriorityUp = { viewModel.increasePriority(it) },
    onPriorityDown = { viewModel.decreasePriority(it) },
    onTaskDone = { viewModel.donePriority(it) },
    onDelete = { viewModel.deleteTask(it) },
    onEdit = {
        val action = TaskListFragmentDirections.actionTaskListFragmentToTaskUpdateFragment(it)
        Navigation.findNavController(requireView()).navigate(action)
    }
)

- `onPriorityUp = { viewModel.increasePriority(it) }`: To jest funkcja, która zostanie przekazana do adaptera i będzie wywoływana, gdy użytkownik kliknie przycisk zwiększający priorytet w widoku elementu listy. Wewnątrz tej lambdy jest wywoływana funkcja `increasePriority` z `viewModel`, a jako argument przekazywane jest obiekt `Task`, który został kliknięty.
- `onPriorityDown = { viewModel.decreasePriority(it) }`: To jest podobna funkcja lambda do poprzedniej, tylko w tym przypadku jest przypisana akcja spadku priorytetu. 
- `onTaskDone = { viewModel.donePriority(it) }`: zostanie wywołana po kliknięciu przycisku *Done* w widoku elementu listy. Wewnątrz tej lambdy jest wywoływana funkcja `donePriority` z `viewModel`.
- `onDelete = { viewModel.deleteTask(it) }`: Podobnie jak poprzednie, ta funkcja lambda obsługuje kliknięcie przycisku *Delete* w widoku elementu listy i wywołuje funkcję `deleteTask` z `viewModel`.
- `onEdit = { ... }`: Tworzony jest obiekt *akcji*, która nawiguje użytkownika do fragmentu aktualizacji zadania z przekazanym identyfikatorem zadania. Następnie jest wywołana nawigacja za pomocą `Navigation.findNavController(requireView()).navigate(action)`.

In [ ]:
viewLifecycleOwner.lifecycleScope.launch {
    viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
        viewModel.tasksState.collectLatest{ taskList ->
            taskAdapter.submitList(taskList)
        }
    }
}

- `viewModel.tasksState` to `Flow` (strumień) danych reprezentujący stan listy zadań.
- `collectLatest` to funkcja kolekcji `Flow`, która nasłuchuje na zmiany w strumieniu danych.
- W momencie, gdy dane w strumieniu `viewModel.tasksState` się zmieniają, blok kodu wewnątrz `collectLatest` zostaje wywołany.
- Wewnątrz tego bloku, otrzymana lista zadań (`taskList`) jest przekazywana do adaptera poprzez `taskAdapter.submitList(taskList)`, co prowadzi do aktualizacji widoku listy zadań w interfejsie użytkownika.

In [ ]:
binding.recycler.apply {
    adapter = taskAdapter
    layoutManager = StaggeredGridLayoutManager(2, StaggeredGridLayoutManager.VERTICAL)
    itemAnimator = null
}

- `layoutManager = StaggeredGridLayoutManager(2, StaggeredGridLayoutManager.VERTICAL)`: Ten wiersz kodu ustawia menedżer układu dla `RecyclerView` na `StaggeredGridLayoutManager`.
- `StaggeredGridLayoutManager` to menedżer układu, który tworzy układ siatki, gdzie elementy w rzędzie mogą mieć różne wysokości (niezależnie od siebie). Dzięki temu uzyskujemy efekt kafelków o nieregularnych wysokościach. W tym przypadku, tworzony jest układ dwukolumnowy (2 oznacza dwie kolumny) z orientacją pionową (`StaggeredGridLayoutManager.VERTICAL`).
- `itemAnimator = null`: Ten wiersz kodu wyłącza domyślny animowany efekt dla zmian elementów w `RecyclerView`.

><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" />

Następnie dodajmy ekran nowego zadania.

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    android:layout_margin="4dp"
    tools:context=".ui.fragments.add.TaskAddFragment">

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/titleInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:hint="Tytuł">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/title"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textShortMessage"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/descriptionInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:hint="Opis">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/description"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textMultiLine"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">

        <RadioGroup
            android:id="@+id/radio"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:orientation="vertical">

            <RadioButton
                android:id="@+id/radioHigh"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="WYSOKI" />

            <RadioButton
                android:id="@+id/radioNormal"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="NORMALNY" />

            <RadioButton
                android:id="@+id/radioLow"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="NISKI" />

            <RadioButton
                android:id="@+id/radioDone"
                android:layout_width="wrap_content"
                android:layout_height="wrap_content"
                android:textSize="24sp"
                android:text="WYKONANY" />
        </RadioGroup>
        </com.google.android.material.card.MaterialCardView>
        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="match_parent"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginEnd="8dp"
            android:layout_marginStart="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">
        <CheckBox
            android:id="@+id/checkbox"
            android:layout_width="match_parent"
            android:layout_gravity="center"
            android:layout_height="match_parent"
            android:text="Czy wykonane"
            android:textSize="20sp"/>
        </com.google.android.material.card.MaterialCardView>

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

    <com.google.android.material.button.MaterialButton
        android:id="@+id/save"
        android:layout_width="0dp"
        android:layout_weight=".5"
        android:layout_height="wrap_content"
        android:textSize="20sp"
        android:layout_marginStart="4dp"
        android:layout_marginEnd="1dp"
        android:text="Zapisz"/>

    <com.google.android.material.button.MaterialButton
        android:id="@+id/back"
        android:layout_width="0dp"
        android:layout_weight=".5"
        android:layout_height="wrap_content"
        android:textSize="20sp"
        android:layout_marginStart="1dp"
        android:layout_marginEnd="4dp"
        style="?attr/materialButtonOutlinedStyle"
        android:text="Wróć"/>
    </LinearLayout>


</LinearLayout>

- `TextInputLayout` dla pola *Tytuł*: Widok obsługujący pole tekstowe dla tytułu zadania. Korzysta z komponentu `TextInputLayout`, który dostarcza etykietę i efekty wizualne do pola tekstowego.
- `TextInputLayout dla pola *Opis*`: Podobnie jak wyżej, ale to pole obsługuje dłuższy tekst, używając typu `textMultiLine` jako rodzaj wprowadzanych danych.
- Grupa `RadioButton`ów: Znajduje się wewnątrz karty (`MaterialCardView`). *Pozwala na wybór priorytetu zadania spośród czterech opcji: Wysoki, Normalny, Niski, Wykonany*.
- `CheckBox` *Czy wykonane*: Druga karta zawiera `CheckBox` umożliwiający zaznaczenie zadania jako wykonane lub nie.
- Przyciski *Zapisz* i *Wróć*: Przyciski do zapisania nowego zadania lub powrotu do poprzedniego ekranu. Ustawione są w linii poziomej, przy użyciu `LinearLayout`.

Przejdźmy do implementacji fragmentu

In [ ]:
class TaskAddFragment : Fragment() {

    private lateinit var binding: FragmentTaskAddBinding

    private val viewModel: TaskViewModel by activityViewModels()

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentTaskAddBinding.inflate(inflater)

        var enabled = true

        binding.apply {
            radioNormal.isChecked = true
            checkbox.setOnClickListener {
                enabled = !enabled
                radio.forEach { (it as RadioButton).isEnabled = enabled }
                if (!enabled) radioDone.isChecked = true
            }


            save.setOnClickListener {
                val title = title.text.toString()
                val description = description.text.toString()

                if ("$title$description".isNotEmpty()){
                    val task = Task(
                        title = title,
                        description = description,
                        isDone = checkbox.isChecked,
                        priority = getPriority()
                    )
                    viewModel.addTask(task)
                    val action = TaskAddFragmentDirections.actionTaskAddFragmentToTaskListFragment()
                    Navigation.findNavController(requireView()).navigate(action)
                }
            }

            back.setOnClickListener {
                val action = TaskAddFragmentDirections.actionTaskAddFragmentToTaskListFragment()
                Navigation.findNavController(requireView()).navigate(action)
            }
        }

        return binding.root
    }

    private fun getPriority(): Priority{
        return when(binding.radio.checkedRadioButtonId){
            binding.radioHigh.id -> Priority.WYSOKI
            binding.radioNormal.id -> Priority.NORMALNY
            binding.radioLow.id -> Priority.NISKI
            else -> Priority.WYKONANY
        }
    }
}

Zmienna `enabled` kontroluje dostępność opcji wyboru priorytetu. Wybrany priorytet inny niż *Wykonany* jest dostępny, a po zaznaczeniu `CheckBoxa`, przestają być dostępne i automatycznie wybierany jest *Wykonany*. 

In [ ]:
checkbox.setOnClickListener {
    enabled = !enabled
    radio.forEach { (it as RadioButton).isEnabled = enabled }
    if (!enabled) radioDone.isChecked = true
}

Pętla `forEach` przechodzi przez każdy element z grupy `radio` (czyli każdy `RadioButton` w układzie). Dla każdego `RadioButtona`, właściwość `isEnabled` jest ustawiana na wartość zmiennej `enabled`.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" />

Ostatnim ekranem będzie ekran aktualizacji zadania, będzie on podobny do ekranu dodawania zadania.

Layout jest bardzo podobny do layoutu ekranu dodawania zadania.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:orientation="vertical"
    tools:context=".ui.fragments.add.TaskAddFragment">

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/titleInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:hint="Tytuł">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/title"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textShortMessage"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <com.google.android.material.textfield.TextInputLayout
        android:id="@+id/descriptionInput"
        style="@style/Widget.MaterialComponents.TextInputLayout.OutlinedBox"
        android:layout_width="match_parent"
        app:boxStrokeColor="@color/black"
        app:boxStrokeWidth="2dp"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:hint="Opis">

        <com.google.android.material.textfield.TextInputEditText
            android:id="@+id/description"
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:layout_marginStart="8dp"
            android:layout_marginTop="20dp"
            android:layout_marginEnd="8dp"
            android:inputType="textMultiLine"
            android:textSize="18sp" />
    </com.google.android.material.textfield.TextInputLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">

            <RadioGroup
                android:id="@+id/radio"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="vertical">

                <RadioButton
                    android:id="@+id/radioHigh"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="WYSOKI" />

                <RadioButton
                    android:id="@+id/radioNormal"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="NORMALNY" />

                <RadioButton
                    android:id="@+id/radioLow"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="NISKI" />

                <RadioButton
                    android:id="@+id/radioDone"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:textSize="24sp"
                    android:text="WYKONANY" />
            </RadioGroup>
        </com.google.android.material.card.MaterialCardView>
        <com.google.android.material.card.MaterialCardView
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="match_parent"
            app:strokeColor="@color/black"
            android:elevation="0dp"
            android:layout_marginEnd="8dp"
            android:layout_marginStart="1dp"
            android:layout_marginTop="2dp"
            app:cardCornerRadius="4dp">
            <CheckBox
                android:id="@+id/checkbox"
                android:layout_width="match_parent"
                android:layout_gravity="center"
                android:layout_height="match_parent"
                android:text="Czy wykonane"
                android:textSize="20sp"/>
        </com.google.android.material.card.MaterialCardView>

    </LinearLayout>

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:orientation="horizontal">

        <com.google.android.material.button.MaterialButton
            android:id="@+id/update"
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            android:textSize="20sp"
            android:layout_marginStart="4dp"
            android:layout_marginEnd="1dp"
            android:text="Aktualizuj"/>

        <com.google.android.material.button.MaterialButton
            android:id="@+id/delete"
            android:layout_width="0dp"
            android:layout_weight=".5"
            android:layout_height="wrap_content"
            android:textSize="20sp"
            android:layout_marginStart="1dp"
            android:layout_marginEnd="4dp"
            android:text="Usuń"/>
    </LinearLayout>
    <com.google.android.material.button.MaterialButton
        android:id="@+id/back"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="20sp"
        android:layout_marginStart="1dp"
        android:layout_marginEnd="4dp"
        style="?attr/materialButtonOutlinedStyle"
        android:text="Wróć"/>


</LinearLayout>

In [ ]:
class TaskUpdateFragment : Fragment() {

    private lateinit var binding: FragmentTaskUpdateBinding

    private val viewModel: TaskViewModel by activityViewModels()

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentTaskUpdateBinding.inflate(inflater)

        var enabled = true

        val id = arguments?.getInt("task_id") ?: 0

        val task = viewModel.getTask(id)

        binding.apply {
            title.setText(task.title)
            description.setText(task.description)
            setRadio(task.priority)
            checkbox.isChecked = task.isDone

            checkbox.setOnClickListener {
                enabled = !enabled
                radio.forEach { (it as RadioButton).isEnabled = enabled }
                if (!enabled) radioDone.isChecked = true
            }

            update.setOnClickListener {
                val title = title.text.toString()
                val description = description.text.toString()

                if ("$title$description".isNotEmpty()){

                    viewModel.updateTask(
                        task.copy(
                        title = title,
                        description = description,
                        isDone = checkbox.isChecked,
                        priority = getPriority()
                        )
                    )
                    val action = TaskUpdateFragmentDirections.actionTaskUpdateFragmentToTaskListFragment()
                    Navigation.findNavController(requireView()).navigate(action)
                }
            }

            back.setOnClickListener {
                val action = TaskUpdateFragmentDirections.actionTaskUpdateFragmentToTaskListFragment()
                Navigation.findNavController(requireView()).navigate(action)
            }

            delete.setOnClickListener {
                viewModel.deleteTask(task)
                val action = TaskUpdateFragmentDirections.actionTaskUpdateFragmentToTaskListFragment()
                Navigation.findNavController(requireView()).navigate(action)
            }
        }

        return binding.root
    }

    private fun setRadio(priority: Priority){
        when(priority){
            Priority.WYSOKI -> binding.radioHigh.isChecked = true
            Priority.NORMALNY -> binding.radioNormal.isChecked = true
            Priority.NISKI -> binding.radioLow.isChecked = true
            else -> binding.radioDone.isChecked = true
        }
    }

    private fun getPriority(): Priority{
        return when(binding.radio.checkedRadioButtonId){
            binding.radioHigh.id -> Priority.WYSOKI
            binding.radioNormal.id -> Priority.NORMALNY
            binding.radioLow.id -> Priority.NISKI
            else -> Priority.WYKONANY
        }
    }
}

Implementacja `UpdateTaskFragment` jest podobna do `AddTaskFragment`, różnicą jest odebranie identyfikatora zadania przekazanego przez `Navigation` z fragmentu `TaskListFragment` i pobranie odpowiadniego zadania z listy dostępnej w `ViewModel`

In [ ]:
val id = arguments?.getInt("task_id") ?: 0

val task = viewModel.getTask(id)

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" />